# Base study

This file contains code to run the evaluation for the base study for all instances

Evaluation runs:
- Baseline with Perses & Vulcan
- SeRu with Perses & Vulcan

## Setup

In [1]:
%use dataframe
%use kandy

In [53]:
import kotlin.io.path.Path
import kotlin.io.path.listDirectoryEntries
import kotlin.io.path.walk
import java.io.IOException
import java.util.concurrent.TimeUnit

val ignoredIssues = listOf("issue_2", "issue_2490")
val instances = File("instances")
    .walk()
    .filter { it.isDirectory && (it.name.startsWith("f") || it.name.startsWith("v")) }
    .filter { ignoredIssues.all { issue -> !it.parent.endsWith(issue) } }
    .map { it.toPath() }

data class CommandOutput(val stdout: String, val stderr: String)

fun String.runCommand(workingDir: File): CommandOutput? {
    return try {
        ProcessBuilder(*split(" ").toTypedArray())
            .directory(workingDir)
            .redirectOutput(ProcessBuilder.Redirect.PIPE)
            .redirectError(ProcessBuilder.Redirect.PIPE)
            .start()
            .also { it.waitFor(60, TimeUnit.MINUTES) }
            .let {
                CommandOutput(
                    stdout = it.inputStream.bufferedReader().readText(),
                    stderr = it.errorStream.bufferedReader().readText()
                )
            }
    } catch (e: IOException) {
        e.printStackTrace()
        null
    }
}

fun String.runCommandInRoot() = runCommand(File(".."))

## Compile SeRu

In [41]:
"go generate ./...".runCommandInRoot()
"go build".runCommandInRoot()

In [54]:
import java.nio.file.Path

fun runSeruIn(folder: Path) = "./seru -i ${folder.pathString}/in.cue -t ${folder.pathString}/test.sh -m".runCommandInRoot()

In [55]:
import kotlin.io.path.*

val results = Path("results").createDirectory()

instances.forEach { instance ->
    val instanceDir = (results / instance).createDirectories()
    val output = runSeruIn(Path("study") / instance)
    
    Path("..")
        .listDirectoryEntries(glob = "seru*")
        .maxBy { it.getLastModifiedTime() }
        .moveTo(instanceDir / "seru_result")
    
    output ?: return@forEach
    (instanceDir / "log.txt").writeBytes(output.stdout.toByteArray())
    (instanceDir / "err.txt").writeBytes(output.stderr.toByteArray())
}